In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from time import time
from datetime import date
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 1000)

# 1 数据预处理

In [19]:
# 载入数据
order = pd.read_csv(
    "../../data/level1/m111-cate-order.csv", sep=','
).rename(columns={'sales_segment1_code': 'category'})

In [20]:
order = order.loc[order.category != '-1']

In [21]:
order_hori = order.set_index(['category', 'order_date']).unstack(level=-1).fillna(0)

In [23]:
order_hori.columns = order_hori.columns.get_level_values(1)

In [25]:
order_hori.to_csv("cate.csv")

In [26]:
order_hori

order_date,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CRJCZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,988.0,1946.0,505.0,458.0,1658.0,3288.0,341.0
CRLJCLQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,551.0,0.0,0.0,0.0,0.0,499.0,0.0,0.0,0.0,0.0,0.0,220.0,0.0,0.0,0.0,220.0,0.0,0.0,1640.0,0.0,1360.0,0.0,0.0,3204.0,7252.0,8.0,8436.0,732.0,340.0,1128.0,168.0,1340.0,2802.0,4840.0,1335.0,1341.0,2238.0,1530.0
CRXDG,0.0,0.0,0.0,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,424.0,662.0,35.0,3197.0,50527.0,20453.0,42618.0,72181.0,23053.0,53622.0,32021.0,36958.0,49208.0,66803.0,70989.0,105633.0,29016.0,70180.0,11693.0,51917.0,26558.0,43637.0,11484.0,9231.0,34593.0,47893.0,37988.0,17996.0,37870.0,38296.0,17546.0
CRXWJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,25.0,0.0,42.0,6579.0,302.0,9813.0,16873.0,23290.0,46326.0,8760.0,15653.0,14458.0,47095.0,39056.0,14983.0,26140.0,46280.0,1808.0,83887.0,3701.0,5643.0,16067.0,7387.0,9912.0,15958.0,34745.0,13749.0,26465.0,12534.0,18907.0
CRYJ,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,276.0,0.0,1.0,633.0,691.0,585.0,10582.0,374860.0,51106.0,103065.0,379267.0,130678.0,209764.0,233844.0,175980.0,284456.0,345838.0,236817.0,819746.0,116237.0,240584.0,62586.0,349711.0,87044.0,152729.0,74881.0,33931.0,154609.0,183317.0,235206.0,148205.0,411438.0,194436.0,79752.0
CRZBQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3779.0,2130.0
CRZJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,0.0,163.0,0.0,129.0,369.0,820.0,846.0,8648.0,361617.0,45819.0,102839.0,313310.0,88643.0,168932.0,158422.0,116557.0,248368.0,278261.0,215035.0,571384.0,221104.0,333583.0,55672.0,351954.0,75867.0,127306.0,51609.0,28937.0,124461.0,177485.0,246097.0,149857.0,376391.0,215436.0,108160.0
CRZTCF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123850.0,0.0,46000.0,42900.0,75300.0,108780.0,49070.0,50090.0,64100.0,85482.0,447090.0,132400.0,167185.0,0.0,223050.0,0.0,90800.0,87103.0,0.0,0.0,0.0,78600.0,136350.0,131198.0,0.0,48500.0
DR,0.0,0.0,0.0,0.0,700.0,0.0,0.0,752.0,0.0,0.0,120.0,0.0,0.0,22752.0,0.0,2741.0,0.0,3000.0,483627.0,719216.0,316858.0,503968.0,503100.0,424378.0,476716.0,522300.0,366051.0,562256.0,1070808.0,484776.0,648238.0,603517.0,943403.0,478800.0,744951.0,518053.0,474784.0,269350.0,434944.0,642922.0,928012.0,922034.0,982956.0,1378460.0,1343216.0,330789.0
